In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2

import gensim

In [2]:
id_texts = [2561, 4, 5, 1111, 522, 3084, 3090, 3343, 2968, 534, 2583, 536, 2565, 2588, 2591, 2084, 2603, 1580, 2606, 
             1072, 1074, 52, 2101, 572, 2622, 63, 2624, 3649, 1602, 587, 588, 78, 1294, 3154, 2134, 599, 89, 1115, 
             1632, 2148, 1125, 2066, 701, 2668, 2067, 3183, 3184, 1137, 2167, 1658, 3708, 1152, 3713, 2692, 133, 
             2184, 3214, 3730, 196, 3927, 3222, 538, 2201, 668, 2206, 161, 680, 1200, 176, 177, 2228, 1717, 2744, 
             3468, 2238, 3775, 3264, 1218, 3268, 710, 3783, 712, 1740, 2224, 1747, 548, 3285, 724, 2682, 3804, 1404, 
             2855, 3925, 1806, 2259, 1779, 3180, 3317, 3318, 1273, 213, 1323, 1281, 812, 1798, 1289, 3850, 2316, 
             3854, 783, 3886, 2323, 3352, 1669, 3357, 1825, 290, 1827, 2343, 1320, 299, 2696, 301, 814, 2353, 
             2357, 822, 311, 1343, 832, 1345, 2370, 1347, 1348, 2374, 2379, 1166, 3300, 2389, 854, 3415, 859, 
             352, 3939, 658, 2144, 879, 880, 1394, 1396, 1911, 377, 2427, 3904, 1406, 384, 897, 903, 1928, 908, 
             1423, 3472, 3473, 1689, 1860, 917, 2456, 3481, 2973, 414, 2467, 421, 1961, 429, 3827, 2480, 945, 441, 
             3829, 1469, 758, 3013, 3526, 3025, 2003, 468, 3030, 1344, 1500, 2526, 3551, 2535, 3562, 665, 3564, 
             1519, 2901, 2200, 510, 1024]

In [3]:
# Read data from file.
with open('/data/share/lab05data/base_19.txt') as f:
    book = f.readlines()
book

['<p>В Группе компаний InfoWatch, которая объединяет ряд российских и зарубежных разработчиков программных продуктов и решений для обеспечения информационной безопасности организаций, противодействия внешним и внутренним угрозам, на конкурсной основе, открыта вакансия <strong>Старший программист C++ (лингвистические технологии).</strong></p> <p><strong>Обязанности:</strong></p> <ul> </ul> <ul> <li>разработка алгоритмов и их реализация на C++;</li> <li>составление технической документации по разрабатываемым компонентам;</li> <li>разработка unit-тестов;</li> <li>разработка, отладка, модификация и оптимизация отдельных сложных компонентов программной системы в соответствии с заданиями, выданными непосредственным руководителем;</li> <li>обзоры кода других разработчиков и рекомендации по его улучшению.</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>знание <strong>С++ </strong>(опыт программирования от 3 лет);</li> <li>знания алгоритмов и структуры данных;</li> <li>высокая 

In [4]:
import glob
# The * is not a regex, it just means "match anything"
# This matches datafile-0.csv, datafile-1.csv, etc.
list_base = glob.glob('/data/share/lab05data/base_*.txt')

In [5]:
glob.glob('/data/share/lab05data/base_*.txt')

['/data/share/lab05data/base_19.txt',
 '/data/share/lab05data/base_3.txt',
 '/data/share/lab05data/base_13.txt',
 '/data/share/lab05data/base_8.txt',
 '/data/share/lab05data/base_1.txt',
 '/data/share/lab05data/base_14.txt',
 '/data/share/lab05data/base_6.txt',
 '/data/share/lab05data/base_12.txt',
 '/data/share/lab05data/base_17.txt',
 '/data/share/lab05data/base_7.txt',
 '/data/share/lab05data/base_10.txt',
 '/data/share/lab05data/base_2.txt',
 '/data/share/lab05data/base_4.txt',
 '/data/share/lab05data/base_15.txt',
 '/data/share/lab05data/base_9.txt',
 '/data/share/lab05data/base_18.txt',
 '/data/share/lab05data/base_16.txt',
 '/data/share/lab05data/base_20.txt',
 '/data/share/lab05data/base_11.txt',
 '/data/share/lab05data/base_5.txt']

In [6]:
# # Turn those into a list of dataframes
# df_list_base = [pd.read_csv(filename) for filename in list_base]
# # df_list_base[:2]

In [7]:
# for dataframe, filename in zip(df_list_base, list_base):
#   dataframe['filename'] = filename

In [ ]:
# dataframe

In [ ]:
# combined_df = pd.concat(df_list_base, ignore_index=True)

In [ ]:
# combined_df 

In [8]:
file_name_and_text = {}
for file in list_base:
    with open (file, "r") as basefile:
        file_name_and_text[file] = basefile.read()
df_list_base = (pd.DataFrame.from_dict(file_name_and_text, orient='index').reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))     

In [ ]:
# print (file_name_and_text)

In [9]:
df_list_base
# df_list_base['id_base'] = df_list_base['file_name'].str.extract('(\d+)').astype(int)
# df_list_base['id_base_1'] = df_list_base['file_name'].str.split('lab05data')[-1].str.extract('(\d+)').astype(int)
# df_list_base
df_list_base['id_base'] = df_list_base['file_name'].str.findall(r'[^a-z/.]+').apply(''.join).str.split('_').str[1]

In [10]:
df_list_base

,file_name,text,id_base
0,/data/share/lab05data/base_19.txt,"<p>В Группе компаний InfoWatch, которая объеди...",19
1,/data/share/lab05data/base_3.txt,"<p>NetCracker Technology Corp., a large softwa...",3
2,/data/share/lab05data/base_13.txt,<p> </p> <p>Компания Wargaming приглашает в св...,13
3,/data/share/lab05data/base_8.txt,<p>Компания Wargaming в лице команды-разработч...,8
4,/data/share/lab05data/base_1.txt,<p>Ищем прекрасного программиста 1С в дружный ...,1
5,/data/share/lab05data/base_14.txt,<p>Разработчик нашей мечты (java developer)</p...,14
6,/data/share/lab05data/base_6.txt,<p><strong>NetCracker Technology</strong></p> ...,6
7,/data/share/lab05data/base_12.txt,<p> </p> <p>Компания Wargaming приглашает в св...,12
8,/data/share/lab05data/base_17.txt,"<p><strong>Группа компаний InfoWatch</strong>,...",17
9,/data/share/lab05data/base_7.txt,"<p>NetCracker Technology Corp., a large softwa...",7


In [11]:
df_list_base.set_index('id_base',inplace=True)
df_list_base.head(5)

,file_name,text
id_base,,
19,/data/share/lab05data/base_19.txt,"<p>В Группе компаний InfoWatch, которая объеди..."
3,/data/share/lab05data/base_3.txt,"<p>NetCracker Technology Corp., a large softwa..."
13,/data/share/lab05data/base_13.txt,<p> </p> <p>Компания Wargaming приглашает в св...
8,/data/share/lab05data/base_8.txt,<p>Компания Wargaming в лице команды-разработч...
1,/data/share/lab05data/base_1.txt,<p>Ищем прекрасного программиста 1С в дружный ...


In [12]:
list_test = []
for i in id_texts:
    list_test.append(f'/data/share/lab05data/test_{i}.txt')
list_test[:5]

['/data/share/lab05data/test_2561.txt',
 '/data/share/lab05data/test_4.txt',
 '/data/share/lab05data/test_5.txt',
 '/data/share/lab05data/test_1111.txt',
 '/data/share/lab05data/test_522.txt']

In [13]:
file_name_and_text_test = {}
for file in list_test:
    with open (file, "r") as testfile:
        file_name_and_text_test[file] = testfile.read()
df_list_test = (pd.DataFrame.from_dict(file_name_and_text_test, orient='index').reset_index()
                .rename(index = str, columns = {'index': 'file_name', 0: 'text'}))     

In [14]:
df_list_test['id_test'] = df_list_test['file_name'].str.findall(r'[^a-z/.]+').apply(''.join).str.split('_').str[1]
df_list_test.head(5)

,file_name,text,id_test
0,/data/share/lab05data/test_2561.txt,<p>Мы ищем талантливых и целеустремленных спец...,2561
1,/data/share/lab05data/test_4.txt,<p><strong>Обязанности</strong>:</p> <ul> <li>...,4
2,/data/share/lab05data/test_5.txt,<p><strong>Обязанности:</strong></p> <ul> <li>...,5
3,/data/share/lab05data/test_1111.txt,<p>В сеть салонов Ваша Оптика требуется продав...,1111
4,/data/share/lab05data/test_522.txt,<p>Компания приглашает специалиста для разовой...,522


In [15]:
df_list_test.set_index('id_test',inplace=True)
df_list_test.head(5)

,file_name,text
id_test,,
2561,/data/share/lab05data/test_2561.txt,<p>Мы ищем талантливых и целеустремленных спец...
4,/data/share/lab05data/test_4.txt,<p><strong>Обязанности</strong>:</p> <ul> <li>...
5,/data/share/lab05data/test_5.txt,<p><strong>Обязанности:</strong></p> <ul> <li>...
1111,/data/share/lab05data/test_1111.txt,<p>В сеть салонов Ваша Оптика требуется продав...
522,/data/share/lab05data/test_522.txt,<p>Компания приглашает специалиста для разовой...


In [16]:
df_list_test.head(5)

,file_name,text
id_test,,
2561,/data/share/lab05data/test_2561.txt,<p>Мы ищем талантливых и целеустремленных спец...
4,/data/share/lab05data/test_4.txt,<p><strong>Обязанности</strong>:</p> <ul> <li>...
5,/data/share/lab05data/test_5.txt,<p><strong>Обязанности:</strong></p> <ul> <li>...
1111,/data/share/lab05data/test_1111.txt,<p>В сеть салонов Ваша Оптика требуется продав...
522,/data/share/lab05data/test_522.txt,<p>Компания приглашает специалиста для разовой...


## Pre-processing

In [17]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /data/home/ekaterina.kucheryavenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/home/ekaterina.kucheryavenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# работа со стоп-словам
from nltk.corpus import stopwords
stop_en = set(stopwords.words('english'))
stop_ru = set(stopwords.words('russian'))
stop_words = set(['p','li','ul','url', 'компания', 'стать', 'зарплата', 'зарп'])
stop_words.update(stop_en,stop_ru)
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words and len(word)>2])

In [19]:
len(stop_en)
len(stop_ru)
len(stop_words)

338

In [30]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup             

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(df_list_test.text[0])  

# Print the raw review and then the output of get_text(), for 
# comparison
print (df_list_test.text[0])
print (example1.get_text())

<p>Мы ищем талантливых и целеустремленных специалистов для работы в Санкт-Петербургском центре разработок.</p> <p><strong>Вам предстоит:</strong></p> <ul> <li>Разрабатывать и сопровождать систему автоматической установки и конфигурирования программных сред.</li> <li>Проектировать архитектуру, выбирать программные средства.</li> <li>Работать с отделами разработки, тестирования и администрирования: сбор требований, расширение функционала системы, выпуск новых версий.</li> </ul> <p> </p> <p><strong>Наши ожидания:</strong></p> <ul> <li>Опыт разработки программного обеспечения от 5-ти лет.</li> <li>Опыт разработки back-end систем на Python.</li> <li>Опыт разработки web-интерфейсов с использованием HTML, JS, CSS, Python, Django.</li> <li>Опыт работы с СУБД: PostgreSQL.</li> <li>Владение объектно-ориентированной методологией, шаблонами проектирования (OOD Patterns).</li> <li>Опыт разработки High Availability систем.</li> <li>English - intermediate.</li> </ul> <p> </p> <p><strong>Мы предлагаем

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda/envs/bd9/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [32]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Zа-яА-Я]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print (letters_only)

Мы ищем талантливых и целеустремленных специалистов для работы в Санкт Петербургском центре разработок  Вам предстоит   Разрабатывать и сопровождать систему автоматической установки и конфигурирования программных сред  Проектировать архитектуру  выбирать программные средства  Работать с отделами разработки  тестирования и администрирования  сбор требований  расширение функционала системы  выпуск новых версий     Наши ожидания   Опыт разработки программного обеспечения от   ти лет  Опыт разработки back end систем на Python  Опыт разработки web интерфейсов с использованием HTML  JS  CSS  Python  Django  Опыт работы с СУБД  PostgreSQL  Владение объектно ориентированной методологией  шаблонами проектирования  OOD Patterns   Опыт разработки High Availability систем  English   intermediate     Мы предлагаем   Индексируемую заработную плату       оплату больничного  Отпуск    календарных дней  оплачиваемый      в соответствии с текущей ставкой  Медицинское сопровождение  ДМС  офисный врач  ст

In [33]:
def preprocess_text(text):
    text = BeautifulSoup(text)
    text = text.get_text()
    text = re.sub("[^a-zA-Zа-яА-Я]", " ", text) 
    # Tokenise words while ignoring punctuation
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols
    
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text.lower())
    
    # Remove stopwords
    keywords= [token for token in tokens if token not in stop_words and len(token)>3]
    
    morph = pymorphy2.MorphAnalyzer()
    lemmas = [morph.parse(keyword)[0].normal_form for keyword in keywords]
    
    lemmatiser = WordNetLemmatizer()
    words = [lemmatiser.lemmatize(token) for token in lemmas]
    
    return words

# import re
# GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
# def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
#     return [t for t in _split(text.lower()) if t and not t.isspace()]

# def token_r(text):
#     words = simple_word_tokenize(text)
#     return [m.parse(x)[0].normal_form for x in words if len(x) >= 4]

# import pymorphy2
# m = pymorphy2.MorphAnalyzer()

In [ ]:
# a = preprocess_text(df_list_base.text[1])
# type(a)

In [21]:
from tqdm import tqdm_notebook as tqdm
test_set = [preprocess_text(txt) for txt in tqdm(df_list_test.text)]

In [ ]:
test_set

In [ ]:
df_1 = df_list_test.text[0]
# test_set = [preprocess_text(txt) for txt in tqdm(df_list_test.text[0])]
test_set = preprocess_text(df_1)
# test_set

In [ ]:
# # создание словаря из уникальных слов
# wrd_index = gensim.corpora.Dictionary(bow_txts)
# print(wrd_index .token2id)

In [ ]:
# no_above = 0.5
# buzzwords = {}
# for key, val in wrd_index.items():
#   if wrd_index.dfs[key] > no_above * wrd_index.num_docs:
#     buzzwords[val] = wrd_index.dfs[key]/wrd_index.num_docs
# wrd_index.filter_extremes(no_above=no_above, no_below=4, keep_n = None)

In [ ]:
# corpus = [wrd_index.doc2bow(txt) for txt in tqdm(bow_txts)]
# corpus

This function will determine which sentences to extract from the article's text by finding the cosine similarity between all tf-idf transformed sentences. The extracted sentences will have the highest average cosine similarity to the remaining sentences. By doing this, the summary should include sentences that show the highest importance to the article.

In [ ]:
# def find_similarities(text):
#     sentences = preprocess_text(text)
#     #vectorize sentences and remove stop words
#     vectorizer = TfidfVectorizer()
#     #transform using TFIDF vectorizer
#     trsfm=vectorizer.fit_transform(sentences)
    
#     #creat df for input article
#     text_df = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=sentences)
    
#     #declare how many sentences to use in summary
#     num_sentences = text_df.shape[0]
#     num_summary_sentences = int(np.ceil(num_sentences**.5))
        
#     #find cosine similarity for all sentence pairs
#     similarities = cosine_similarity(trsfm, trsfm)
    
#     #create list to hold avg cosine similarities for each sentence
#     avgs = []
#     for i in similarities:
#         avgs.append(i.mean())
     
#     #find index values of the sentences to be used for summary
#     top_idx = np.argsort(avgs)[-num_summary_sentences:]
    
#     return top_idx

In [ ]:
# # Scikit Learn
# from sklearn.feature_extraction.text import CountVectorizer
# import pandas as pd

# # Create the Document Term Matrix
# count_vectorizer = CountVectorizer()
# sparse_matrix = count_vectorizer.fit_transform(df_list_base.text)
# # Similarity between the first document (“Alpine snow winter boots”) with each of the other documents of the set:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(sparse_matrix[0:1], sparse_matrix)
# array([[ 0.50305744 ,  0.16651513,  0.62305744,  0.13448867]])

In [ ]:
# df_list_test.text[0].index


In [48]:
def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """

    train_tfidf = tokenizer.fit_transform(train_series)
    new_tfidf = tokenizer.transform(new_series)
    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score

train_set = pd.Series(df_list_base.text)
test_set = pd.Series(df_list_test.text)
tokenizer = TfidfVectorizer(analyzer=preprocess_text) # initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
score.head(10)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda/envs/bd9/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,ix_new,ix_train,score
0,2561,19,0.241531
1,4,19,0.211504
2,5,19,0.297733
3,1111,19,0.107998
4,522,19,0.101888
5,3084,19,0.090757
6,3090,19,0.193223
7,3343,19,0.174380
8,2968,19,0.126408
9,534,19,0.284497


In [49]:
score.head(300)

,ix_new,ix_train,score
0,2561,19,0.241531
1,4,19,0.211504
2,5,19,0.297733
3,1111,19,0.107998
4,522,19,0.101888
5,3084,19,0.090757
6,3090,19,0.193223
7,3343,19,0.174380
8,2968,19,0.126408
9,534,19,0.284497


In [35]:
df_cos = score.groupby(['ix_new']).sum()
df_cos.head(10)

,score
ix_new,
1024,1.613293
1072,2.996213
1074,2.353618
1111,1.252495
1115,1.767176
1125,3.887458
1137,3.091197
1152,3.650625
1166,1.658839


In [36]:
# Находим среднее значение
m = df_cos['score'].mean()
m

2.558896108000889

In [37]:
defined = (df_cos[df_cos['score']>=m].index.values.astype(int))
defined.shape

(96,)

In [38]:
other = (df_cos[df_cos['score']<m].index.values.astype(int))

In [39]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [40]:
numpyData = {"defined": defined, "other": other}
# encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
with open("../lab05.json", "w") as write_file:
    json.dump(numpyData, write_file, cls=NumpyArrayEncoder)

In [41]:
# import logging
# logging.basicConfig(format='%(asctime)s : &(levelname)s : %(message)s', level=logging.INFO)


In [ ]:
# Create an instance of TfidfVectorizer
# vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# # Fit to the data and transform to feature matrix
# X_train = vectoriser.fit_transform(X_train['speech'])
# # Convert sparse matrix to dataframe
# X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# # Save mapping on which index refers to which words
# col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
# for col in X_train.columns:
#     X_train.rename(columns={col: col_map[col]}, inplace=True)
# X_train